In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from src.PreProc_Data.DataProc import StackedSequenceDataset
import numpy as np
from src.MZA_Experiment import MZA_Experiment
# from MZA_Experiment import MZA_Experiment

In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps") 
else:
    device = torch.device("cpu")

In [3]:
tensor = torch.tensor([
    [0, 0, 0, 0],
    [1, 2, 3, 4],
    [0, 0, 0, 0],
    [0, 1, 0, 1]
])

# Check if all elements in each row are zero
all_zeros_rows = torch.all(tensor == 0, dim=1)

# Print the result
print("Rows containing all zeros:", all_zeros_rows)







Rows containing all zeros: tensor([ True, False,  True, False])


In [16]:
tensor = torch.tensor([
    [[0, 0],
     [0, 0]],

    [[1, 2],
     [0, 0]],

    [[0, 0],
     [0, 0]]
])

t = tensor
for i in range(tensor.ndim-1):
    if i == 0:
        t = torch.all(t == 0, dim = -1)
    else:
        t = torch.all(t == True, dim = -1)
~t 

tensor[~t]

tensor([[[1, 2],
         [0, 0]]])

In [12]:
torch.all(tensor == 0, dim = -1)

tensor([[ True,  True],
        [False,  True],
        [ True,  True]])

In [3]:
data = np.load("Data/2DCylinder/processed_data/npyfiles/nektar_cyl_data_20_dt0.25_T200.npy")
# data = data[:,:,:2]
# data = np.einsum('ijk->jki',data)
data.shape


(1, 801, 7938)

In [5]:
args_dict = {"device": device, "seq_len": 5, "pred_horizon": 5}
dataset   = StackedSequenceDataset(data, args_dict)
it = iter(dataset)
Phi_seq, Phi_nn = next(it)
print("Phi_seq shape:", Phi_seq.shape)
print("Phi_seq shpae:", Phi_nn.shape)
print(len(dataset))

0  data1:  torch.Size([1, 5, 7938])
1  data1:  torch.Size([1, 5, 7938])
2  data1:  torch.Size([1, 5, 7938])
3  data1:  torch.Size([1, 5, 7938])
4  data1:  torch.Size([1, 5, 7938])
5  data1:  torch.Size([1, 5, 7938])
6  data1:  torch.Size([1, 5, 7938])
7  data1:  torch.Size([1, 5, 7938])
8  data1:  torch.Size([1, 5, 7938])
9  data1:  torch.Size([1, 5, 7938])
10  data1:  torch.Size([1, 5, 7938])
11  data1:  torch.Size([1, 5, 7938])
12  data1:  torch.Size([1, 5, 7938])
13  data1:  torch.Size([1, 5, 7938])
14  data1:  torch.Size([1, 5, 7938])
15  data1:  torch.Size([1, 5, 7938])
16  data1:  torch.Size([1, 5, 7938])
17  data1:  torch.Size([1, 5, 7938])
18  data1:  torch.Size([1, 5, 7938])
19  data1:  torch.Size([1, 5, 7938])
20  data1:  torch.Size([1, 5, 7938])
21  data1:  torch.Size([1, 5, 7938])
22  data1:  torch.Size([1, 5, 7938])
23  data1:  torch.Size([1, 5, 7938])
24  data1:  torch.Size([1, 5, 7938])
25  data1:  torch.Size([1, 5, 7938])
26  data1:  torch.Size([1, 5, 7938])
27  data1: 

: 

In [ ]:
seqdataset = SequenceDataset(data, device)
# dataloader = DataLoader(dataset, batch_size = 16, shuffle =True)

In [ ]:
it = iter(seqdataset)
Phi_seq, Phi_nn = next(it)
for i, data in enumerate(seqdataset):
    if (i!=0):
        Phi_seq = torch.cat((Phi_seq, data[0]), dim = 0)
        Phi_nn  = torch.cat((Phi_nn, data[1]), dim = 0)
print(Phi_seq.shape)
print(Phi_nn.shape)
   


In [ ]:
dataloader = DataLoader(dataset, batch_size = 16, shuffle =True)

it = iter(dataloader)
Phi_seq, Phi_nn = next(it)
print(Phi_seq.shape)
print(Phi_nn.shape)

In [ ]:
k = {"a":1,"b":2}
print(type(k))

In [ ]:
from Layers.Koopman import Koopman


In [ ]:
koopman = Koopman(5,device)
koopman.kMatrixUT.requires_grad 

In [ ]:
sum(p.numel() for p in koopman.parameters())

In [ ]:
for parameter in koopman.parameters():
    print(parameter)

In [ ]:
import random
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F

seq_len = 10
n_class = 5
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.W = nn.Parameter(torch.randn(seq_len, n_class), requires_grad=True)
        
    def forward(self, x):
        x = torch.matmul(x, self.W)
        x = F.relu(x)
        x = F.log_softmax(x)
        return x

model = SimpleNet()
print('parameters----------')
for parameter in model.parameters():
    print("hello")
    print(parameter)
    print("hello")

optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.0001)
loss_fn = nn.NLLLoss()
for i in range(10000):
    in_data = Variable(torch.FloatTensor([[random.randint(0, n_class-1) for _ in range(seq_len)]]))
    target_data = Variable(torch.LongTensor([ int(in_data.data[0][0]) ]))
    
    optimizer.zero_grad()
    output = model(in_data)
    loss = loss_fn(output, target_data)
    loss.backward()
    optimizer.step()
    if i % 1000 == 0:
        print('loss:', loss.data[0])
    
# test
print('======Test======')
for i in range(10):
    in_data = Variable(torch.FloatTensor([[random.randint(0, n_class-1) for _ in range(seq_len)]]))
    target_data = int(in_data.data[0][0])
    out = model(in_data)
    _, ind = torch.max(out.data[0], 0)
    print(ind[0] == target_data, 'out', ind[0], 'label', target_data)

print('after parameters')
print(next(model.parameters()))

In [ ]:
import numpy as np
data = np.load("Data/KS/ks_N256_dt0.001_L6.npy")
data = data[:15,::100,:]
data.shape

In [ ]:
np.save("Data/KS/ks_N256_dt0.001_L6_short.npy", data)

In [ ]:
a = [0,1,2,3,4,5,6,7]
print(a[5:])

import torch

b = torch.tensor(a)

tuple(b.shape)

In [15]:
b = torch.tensor([[1,2],[3,4]]).reshape(2,2)

a = torch.arange(1,9).reshape(2,2,2)

c = torch.einsum("bsh,bs->bh",[a,b])
print(a.shape,b.shape,c.shape)
a,b,c

torch.Size([2, 2, 2]) torch.Size([2, 2]) torch.Size([2, 2])


(tensor([[[1, 2],
          [3, 4]],
 
         [[5, 6],
          [7, 8]]]),
 tensor([[1, 2],
         [3, 4]]),
 tensor([[ 7, 10],
         [43, 50]]))